En el análisis realizado, concluimos la siguiente información:
- Tenemos como entrada (x) 200.000 datos de transacciones (big data).
- No hay transacciones duplicadas ni inválidas.
- Tenemos como salida (y), una clasificación binaria (0,1): fraude o no (columna Is_fraud).
- Tenemos un total de 10088 transacciones etiquetadas comno fraudes.
- Al estar utilizando un dataset ya existente, el etiquetado realizado es correcto y no necesita de un trabajo de etiquetado adicional.
- Tenemos diferentes tipos de transacciones: crédito, débito, facturas, transferencia, y retiradas.

Una vez realizado este análisis, comenzamos con el preprocesamiento de los datos, con el objetivo de establecer un "Data Pipeline". A continuación, se detalla el Data Pipeline definido para preparar los datos antes de comenzar con el modelado ML.


# 1. Limpieza de datos
Ya hemos confirmado que no tenemos valores nulos ni duplicados, por lo que procedemos con la transformación de datos.

Vamos a eliminar columnas irrelevantes, es decir, columnas que no aportan información útil:
- Custome_ID, Customer_Name, Customer_Contact, Customer_Email, Transaction_ID, Merchant_ID. Estos son identificadores no predictivos.
- Transaction_Description. Texto difícil de utilizar sin procesamiento NLP.
- Transaction_Location. Tenemos ya almacenada información del lugar.

In [71]:
import pandas as pd

In [72]:
df = pd.read_csv("data/raw_data.csv")

In [73]:
print(df.columns)


Index(['Customer_ID', 'Customer_Name', 'Gender', 'Age', 'State', 'City',
       'Bank_Branch', 'Account_Type', 'Transaction_ID', 'Transaction_Date',
       'Transaction_Time', 'Transaction_Amount', 'Merchant_ID',
       'Transaction_Type', 'Merchant_Category', 'Account_Balance',
       'Transaction_Device', 'Transaction_Location', 'Device_Type', 'Is_Fraud',
       'Transaction_Currency', 'Customer_Contact', 'Transaction_Description',
       'Customer_Email'],
      dtype='object')


In [74]:
drop_columns = ["Customer_ID", "Customer_Name", "Customer_Contact", "Customer_Email", "Transaction_ID", "Merchant_ID","Transaction_Description", "Transaction_Location"]
df.drop(columns=drop_columns, inplace=True)
print(df.columns)

Index(['Gender', 'Age', 'State', 'City', 'Bank_Branch', 'Account_Type',
       'Transaction_Date', 'Transaction_Time', 'Transaction_Amount',
       'Transaction_Type', 'Merchant_Category', 'Account_Balance',
       'Transaction_Device', 'Device_Type', 'Is_Fraud',
       'Transaction_Currency'],
      dtype='object')


# 2. Codificación de variables categóricas
Vamos a convertir columnas categóricas a valores numéricos, ya que generalmente los algoritmos de ML trabajan mejor con valores numéricos. Vamos a aplicar los siguientes métodos:
- one-hot encoding: vamos a crear una nueva columna para cada categoría y usar valores 0 y 1. Esto lo haremos en aquellas variables que tienen pocas categorías: Gender, Account_Type, Transaction_Type, Merchant_Category, Device_Type, Transaction_Currency.
- Label encoding: vamos a convertir cada categoría en un número entero. Esto lo aplicaremos en variables con muchas categorías: City, State, Bank_Branch, Transaction_Device.

In [75]:
df = pd.get_dummies(df, columns=["Gender", "Account_Type","Transaction_Type", "Merchant_Category", "Device_Type", "Transaction_Currency"], drop_first=True)
print(df.columns)

Index(['Age', 'State', 'City', 'Bank_Branch', 'Transaction_Date',
       'Transaction_Time', 'Transaction_Amount', 'Account_Balance',
       'Transaction_Device', 'Is_Fraud', 'Gender_Male',
       'Account_Type_Checking', 'Account_Type_Savings',
       'Transaction_Type_Credit', 'Transaction_Type_Debit',
       'Transaction_Type_Transfer', 'Transaction_Type_Withdrawal',
       'Merchant_Category_Electronics', 'Merchant_Category_Entertainment',
       'Merchant_Category_Groceries', 'Merchant_Category_Health',
       'Merchant_Category_Restaurant', 'Device_Type_Desktop',
       'Device_Type_Mobile', 'Device_Type_POS'],
      dtype='object')


In [76]:
from sklearn.preprocessing import LabelEncoder

for col in ["City", "State", "Bank_Branch", "Transaction_Device"]:
    df[col] = LabelEncoder().fit_transform(df[col])

In [77]:
df.head()

,Age,State,City,Bank_Branch,Transaction_Date,Transaction_Time,Transaction_Amount,Account_Balance,Transaction_Device,Is_Fraud,...,Transaction_Type_Transfer,Transaction_Type_Withdrawal,Merchant_Category_Electronics,Merchant_Category_Entertainment,Merchant_Category_Groceries,Merchant_Category_Health,Merchant_Category_Restaurant,Device_Type_Desktop,Device_Type_Mobile,Device_Type_POS
0,60,15,127,127,23-01-2025,16:04:07,32415.45,74557.27,17,0,...,True,False,False,False,False,False,True,False,False,True
1,51,18,100,100,11-01-2025,17:14:53,43622.60,74622.66,9,0,...,False,False,False,False,False,False,True,True,False,False
2,20,4,13,13,25-01-2025,03:09:52,63062.56,66817.99,0,0,...,False,False,False,False,True,False,False,True,False,False
3,57,28,22,22,19-01-2025,12:27:02,14000.72,58177.08,8,0,...,False,False,False,True,False,False,False,False,True,False
4,43,25,7,7,30-01-2025,18:30:46,18335.16,16108.56,16,0,...,True,False,False,True,False,False,False,False,True,False


# 3. Transformar fechas y tiempos
Lo siguiente será transformar los datos fechas y horas en características numéricas.

In [78]:
df["Transaction_Date"] = pd.to_datetime(df["Transaction_Date"])
df["Transaction_Time"] = pd.to_datetime(df["Transaction_Time"], format="%H:%M:%S")

# Extraer información útil
df["Year"] = df["Transaction_Date"].dt.year
df["Month"] = df["Transaction_Date"].dt.month
df["Day"] = df["Transaction_Date"].dt.day
df["DayOfWeek"] = df["Transaction_Date"].dt.dayofweek
df["Hour"] = df["Transaction_Time"].dt.hour

# Eliminar las columnas originales
df.drop(columns=["Transaction_Date", "Transaction_Time"], inplace=True)

df.head()

C:\Users\javi2\AppData\Local\Temp\ipykernel_8828\1101040754.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["Transaction_Date"] = pd.to_datetime(df["Transaction_Date"])


,Age,State,City,Bank_Branch,Transaction_Amount,Account_Balance,Transaction_Device,Is_Fraud,Gender_Male,Account_Type_Checking,...,Merchant_Category_Health,Merchant_Category_Restaurant,Device_Type_Desktop,Device_Type_Mobile,Device_Type_POS,Year,Month,Day,DayOfWeek,Hour
0,60,15,127,127,32415.45,74557.27,17,0,True,False,...,False,True,False,False,True,2025,1,23,3,16
1,51,18,100,100,43622.60,74622.66,9,0,False,False,...,False,True,True,False,False,2025,1,11,5,17
2,20,4,13,13,63062.56,66817.99,0,0,True,False,...,False,False,True,False,False,2025,1,25,5,3
3,57,28,22,22,14000.72,58177.08,8,0,False,False,...,False,False,False,True,False,2025,1,19,6,12
4,43,25,7,7,18335.16,16108.56,16,0,False,False,...,False,False,False,True,False,2025,1,30,3,18


# 4. Outliers
Los valores extremos normalmente afectan al rendimiento de los modelos, especialmente en dsitintas variables. Para ello, vamos a detectar los outliers y a limitarlos, sin perder información.

A falta de analizar los resultados del modelo, podría ser que hayan transacciones anormalmente grandes, y saldos de cuentas muy altos o muy bajos. Estos valores extremos pueden afectar al entrenamiento de los modelos de ML, dominando la escala de los datos y afectando al rendimiento del modelo. No obstante, eliminar los outliers no será una solución ya podremos perder información valiosa. Para ello, existen diferentes técnicas para reducir el impacto de valores extremos sin afectar demasiado a los datos. En nuestro caso, proponemos utilizar Winsorrization, aunque esto deberá ser estudiado en la fase de entrenamiento.

Este método se basa en recortar los valores extremos y reemplazarlos por valores cercanos. Proponemos inicialmente recortar el 1% superior e inferior.

In [79]:
from scipy.stats.mstats import winsorize

# Aplicamos Winsorization a las variables más afectadas por outliers
df["Transaction_Amount"] = winsorize(df["Transaction_Amount"], limits=[0.01, 0.01])  # Recorta el 1% superior e inferior
df["Account_Balance"] = winsorize(df["Account_Balance"], limits=[0.01, 0.01])


# 5. Normalización y escalado
Muchos modelos de ML son sensibles a la escala de datos. Con el objetivo en mente de mejorar el rendimiento vamos a escalar los datos con StandardScaler (aunque existen más técnicas) por su sencillez. En el caso de que en posteriores análisis detectemos muchos outliers, podríamos implementar un RobustScaler. Otra técnica muy utilizada es MinMaxScaler.

Aplicando StandardScaler en las variables numéricas, vamos a mejorar la convergencia del modeloy evitar que una variable domine sobre otras posibles.

In [80]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

num_features = ["Age", "Transaction_Amount", "Account_Balance", "Year", "Month", "Day", "DayOfWeek", "Hour"]

df[num_features] = scaler.fit_transform(df[num_features])


Finalmente, vamos a almacenar el conjunto de datos (en formato .csv) preprocesado y limpiado.

In [81]:
df.to_csv("data/dataset.csv", index=False)